**PROBLEM STATEMENT**

**Develop a prototype of a video conversion process that can convert SD resolution (640 x 480px)
videos to HD resolution (1280 x 720px) videos using a diffusion type model. The tool should be able
to fill in the blank areas on the left and right sides of the video with relevant pixels or image parts
while preserving the context of the video.**

**1. Install necessary packages**

  We've first installed the required Python libraries: opencv-python-headless for handling video processing, diffusers for the diffusion model, pillow for image manipulation, and torch for PyTorch operations.

In [ ]:
!pip install opencv-python-headless diffusers pillow torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

**2. Import libraries and define frameextract function**

 We've imported the OpenCV and NumPy libraries

 We're defining frameextract, a function that extracts frames from a video file. It reads frames using OpenCV's VideoCapture and appends each frame to a list until the end of the video. The list of frames is then returned.

In [ ]:
import cv2
import numpy as np
def frameextract(video_path):
  cv=cv2.VideoCapture(video_path)
  list_of_frame=[]
  while cv.isOpened():
    ret,frame=cv.read()
    if not ret:
      break
    else:
      list_of_frame.append(frame)
  cv.release()
  return list_of_frame

**3. Import libraries**

 In the next cell, we've imported the torch library, the StableDiffusionInpaintPipeline from the diffusers library, and the Image and ImageDraw classes from the PIL library.


In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image,ImageDraw,ImageDraw2

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

**4. Load the Stable Diffusion model**

 We're loading the Stable Diffusion inpainting model using the from_pretrained method and setting the data type to float16 for efficiency.

 We're moving the model to the GPU if available, otherwise using the CPU.

In [ ]:
model_name= "runwayml/stable-diffusion-inpainting"
pipe=StableDiffusionInpaintPipeline.from_pretrained(model_name,torch_dtype=torch.float16)
pipe=pipe.to('cuda' if torch.cuda.is_available() else 'cpu')
# pipe.to('cuda11')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

text_encoder/model.safetensors not found


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


**5. Define inpaint_pipeline function**

 Now, We've defined an inpaint_pipeline, a function that takes an image and the inpainting model pipeline as input.

 We will resize the input image to 512x512 and converting it to PIL format.

 We're creating a mask with two filled rectangles on the left and right sides, representing the areas to be inpainted.

 The inpainting model is used to generate the inpainted image based on the input image and mask.

 We're converting the inpainted image back to OpenCV format and replacing the center of the inpainted image with the original center.

 The final image is resized  to HD resolution (1280x720) and returned.



In [ ]:


def inpaint_pipeline(img,pipe):
  #Lets resize image first
  resized_image=cv2.resize(img,(512,512))
  converted_to_pil_format=Image.fromarray(cv2.cvtColor(resized_image,cv2.COLOR_BGR2RGB))

  #Mask
  mask1=Image.new('L',(512,512),0)
  drawing_var=ImageDraw.Draw(mask1)
  drawing_var.rectangle([(0,0),(128,512)],fill=255)
  drawing_var.rectangle([(384,0),(512,512)],fill=255)
  # inpainted_image=pipe(prompt=,)/
  inpaint_image=pipe(prompt="",image=converted_to_pil_format,mask_image=mask1).images[0]

  inpaint_image_cv=cv2.cvtColor(np.array(inpaint_image),cv2.COLOR_RGB2BGR)

  #Changing the center value to original value
  center_original=resized_image[:,128:384]
  inpaint_image_cv[:,128:384]=center_original
  hd_image=cv2.resize(inpaint_image_cv,(1280,720))
  return hd_image

**6. Install accelerator**

We've installed the accelerate library, which helps in accelerating PyTorch code.

In [ ]:
!pip install accelerator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 5.5 MB/s eta 0:00:00


**7. Define lets_processframe function**

 A simple function lets_processframe processes a single frame using the inpaint_pipeline function and returns the inpainted frame.

In [ ]:
def lets_processframe(frame,pipe):
  return inpaint_pipeline(frame,pipe)



**8. Import libraries and define processframes function**

 We've imported ThreadPoolExecutor for parallel processing and tqdm for progress bars.

 We're defining processframes, a function that processes multiple frames in parallel using threading. It utilizes a thread pool to process frames concurrently and displays a progress bar.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def processframes(frames,pipe,threads=4):
  with ThreadPoolExecutor(max_workers=threads) as raj:
     processed_frames=list(tqdm(raj.map(lambda frame:lets_processframe(frame,pipe),frames),total=len(frames)))
  return processed_frames


**9. Define lets_video_saved function**

  Defined lets_video_saved, a function that saves a list of frames as a video file.

  We're creating a VideoWriter object with the specified path, codec, FPS, and frame size, then writing each frame to the video and releasing the VideoWriter.

In [ ]:
def lets_video_saved(frames,path,fps):
  height,width,_=frames[0].shape
  video=cv2.VideoWriter(path,cv2.VideoWriter_fourcc(*'mp4v'),fps,(width,height))
  for frame in frames:
    video.write(frame)
  video.release()

**10. Define final_convert_video function**

 We've defined final_convert_video, a function that converts an input video to HD resolution.

 We're capturing the video to get its FPS and extracting all frames.

 We're processing frames in batches and saving the processed frames as a new video.

In [ ]:
def final_convert_video(input_path,output_path_video):
  cv=cv2.VideoCapture(input_path)
  #Lets get its fps
  fps=cv.get(cv2.CAP_PROP_FPS)
  cv.release()
  frames=frameextract(input_path)
  # print(frames)
  batch_size=2
  processed_frames=[]
  for i in tqdm(range(0,len(frames),batch_size),desc='Processing'):
    batch=frames[i:i+batch_size]
    # print(batch)
    processed_batch=processframes(batch,pipe)
    processed_frames.extend(processed_batch)
  lets_video_saved(processed_frames,output_path_video,fps)

**11. Main function to run the conversion**

We're setting the input and output video paths and calling final_convert_video to perform the conversion.

In [ ]:



if __name__=="__main__":
  input_path="input_video.mp4"
  output_path="output_raj.mp4"
  final_convert_video(input_path,output_path)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.

Processing: 100%|██████████| 13/13 [03:35<00:00, 16.58s/it]
